In [1]:
pip install transformers torch requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
import torch
import requests
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [13]:
from transformers import BertTokenizer,BertModel
import pandas as pd
import torch
import numpy as np


In [14]:
model=BertModel.from_pretrained("bert-base-uncased",output_hidden_states=True)
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
def bert_text_prepare(text,tokenizer):
  marked_text="[CLS]"+text+"[SEP]"
  tokenized_text=tokenizer.tokenize(marked_text)
  indexed_tokens=tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids=[1]*len(indexed_tokens)
  #convert inputs to pytorch sensors:
  tokens_tensor=torch.tensor([indexed_tokens])
  segment_tensors=torch.tensor([segments_ids])
  return tokenized_text,tokens_tensor,segment_tensors

In [17]:
#convert input into embeddings
def get_embeddings(tokens_tensor,segment_tensors,model):
  with torch.no_grad():
    outputs=model(tokens_tensor,segment_tensors)
    hidden_states=outputs[2][1:]
  token_embeddings=hidden_states[-1]
  token_embeddings=torch.squeeze(token_embeddings,dim=0)
  list_token_embeddings=[token_embed for token_embed in token_embeddings]
  return list_token_embeddings

In [20]:
word_embeddings=[]
for text in texts:
  tokenized_text,tokens_tensor,segment_tensors=bert_text_prepare(text,tokenizer)
  list_token_embeddings=get_embeddings(tokens_tensor,segment_tensors,model)
  bank_index=tokenized_text.index('bank')
  bank_embedding=list_token_embeddings[bank_index]
  word_embeddings.append(bank_embedding)


In [19]:
texts=["bank",
       "the river bank was flooded",
       "the bank was out of money"]

In [21]:
from scipy.spatial.distance import cosine
list_of_distances=[]
for text1,embed1 in zip(texts,word_embeddings):
  for text2,embed2 in zip(texts,word_embeddings):
    cos=1-cosine(embed1,embed2)
    list_of_distances.append((text1,text2,cos))

distances_df=pd.DataFrame(list_of_distances,columns=['text1','text2','distance'])




In [22]:
distances_df

,text1,text2,distance
0,bank,bank,1.000000
1,bank,the river bank was flooded,0.325561
2,bank,the bank was out of money,0.491493
3,the river bank was flooded,bank,0.325561
4,the river bank was flooded,the river bank was flooded,1.000000
5,the river bank was flooded,the bank was out of money,0.533799
6,the bank was out of money,bank,0.491493
7,the bank was out of money,the river bank was flooded,0.533799
8,the bank was out of money,the bank was out of money,1.000000
